In [1]:
import os
import os.path as osp

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.nn import GCNConv
import numpy as np
import torch
from torch.nn import Sequential, Linear, ReLU
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, accuracy_score

from utils import (
    get_link_labels,
    prediction_fairness,
)

from torch_geometric.utils import train_test_split_edges

device = "cuda" if torch.cuda.is_available() else "cpu"
import wandb
import tqdm

/opt/conda/envs/graph/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch_geometric.utils import to_undirected, to_networkx, k_hop_subgraph, is_undirected
from torch_geometric.data import Data
from torch_geometric.loader import GraphSAINTRandomWalkSampler
from torch_geometric.seed import seed_everything

In [3]:
from model.gcn import GCN
from model.deletegcn import GCNDelete
from model.Wdeletegcn import WGCNDelete
from model.PNNMask import PGNNMask
import torch.nn as nn

In [4]:
import numpy as np
import torch
from sklearn.multioutput import MultiOutputClassifier
from torch_sparse import SparseTensor
from sklearn.metrics import (
    roc_auc_score,
    make_scorer,
    balanced_accuracy_score,
)
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, pipeline, metrics

# Metrics
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
)
from itertools import combinations_with_replacement

In [5]:
from torch_geometric import seed_everything

seed_everything(1888)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
from training_args import parse_args
args=parse_args()

In [8]:
from model.gcn import GCN

In [9]:
dataset = "citeseer" #"cora" "pubmed"
path = osp.join(osp.dirname(osp.realpath('__file__')), "..", "data", dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())

In [10]:
pwd

'/home/jupyter/FairDrop'

In [11]:
path

'/home/jupyter/FairDrop/../data/citeseer'

In [12]:
test_seeds = [0,1,2,3,4,5]
acc_auc = []
fairness = []
acc_auc_ori = []
fairness_ori = []

In [13]:
data = dataset[0]

In [14]:
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [15]:
args.in_dim=data.num_features

In [16]:
protected_attribute = data.y
Y = torch.LongTensor(protected_attribute).to(device)
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data, val_ratio=0.1, test_ratio=0.2)
data = data.to(device)
num_classes = len(np.unique(protected_attribute))
N = data.num_nodes

/opt/conda/envs/graph/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [17]:
6374/100*3

191.22

In [18]:


Y = torch.LongTensor(protected_attribute).to(device)
Y_diff = (
    Y[data.train_pos_edge_index[0, :]] != Y[data.train_pos_edge_index[1, :]]
).to(device)

Y_same = (
    Y[data.train_pos_edge_index[0, :]] == Y[data.train_pos_edge_index[1, :]]
).to(device)


In [19]:
torch.sum(Y_diff)
torch.sum(Y_same)
diff=Y_diff.nonzero().squeeze()
same=Y_same.nonzero().squeeze()
same

tensor([   0,    1,    2,  ..., 6370, 6371, 6373], device='cuda:0')

In [20]:
edge_to_delete=100
ratio=3
diff_size=int(edge_to_delete*(ratio)/(ratio+1))
same_size=int(edge_to_delete*(1)/(ratio+1))
idx_diff = torch.randperm(diff.shape[0])[:diff_size]
df_diff_idx = diff[idx_diff]
idx_same = torch.randperm(same.shape[0])[:same_size]
df_same_idx = same[idx_same]
df_global_idx=torch.cat((df_diff_idx,df_same_idx),0)

In [21]:
dr_mask = torch.ones(data.train_pos_edge_index.shape[1], dtype=torch.bool)
dr_mask[df_global_idx] = False
dr_mask=dr_mask.to(device)

df_mask = torch.zeros(data.train_pos_edge_index.shape[1], dtype=torch.bool)
df_mask[df_global_idx] = True
df_mask=df_mask.to(device)

In [22]:
###

In [23]:
from itertools import product
full_pos_edge_index=data.train_pos_edge_index
print(full_pos_edge_index.shape)

torch.Size([2, 6374])


In [24]:
is_undirected(full_pos_edge_index)

True

In [25]:
##random add edges

for target in range(edge_to_delete):
    _, l_hop_edge, _, l_hop_mask = k_hop_subgraph(
            [data.train_pos_edge_index[:, df_mask][0,:][target]], 
            1, 
            data.train_pos_edge_index,
            num_nodes=data.num_nodes)

    ldset1=l_hop_edge.flatten().unique()
    ldset1=ldset1[ldset1!=data.train_pos_edge_index[:, df_mask][0,:][target].item()]

    _, r_hop_edge, _, r_hop_mask = k_hop_subgraph(
            [data.train_pos_edge_index[:, df_mask][1,:][target]], 
            1, 
            data.train_pos_edge_index,
            num_nodes=data.num_nodes)

    rdset1=r_hop_edge.flatten().unique()
    rdset1=rdset1[rdset1!=data.train_pos_edge_index[:, df_mask][1,:][target].item()]

    combine = list(product(ldset1.tolist(), rdset1.tolist()))
    ind=[True if Y[a[0]]!=Y[a[1]] else False for a in combine ]
    sele_pair=[combine[i] for i in range(len(ind)) if ind[i]==True]
    n=len(sele_pair)
    if(n>0):
        add=min(n,2)
        add_pair_ind=torch.randperm(len(sele_pair))[:add]
        add_pair=[sele_pair[i] for i in add_pair_ind]
        add_matrix_0=[add_pair[i][0] for i in range(len(add_pair))] 
        add_matrix_1=[add_pair[i][1] for i in range(len(add_pair))] 
        add_matrix1=torch.tensor([add_matrix_0,add_matrix_1]).to(device)
        add_matrix2=torch.tensor([add_matrix_1,add_matrix_0]).to(device)
        full_pos_edge_index=torch.cat((full_pos_edge_index,add_matrix1),1)
        full_pos_edge_index=torch.cat((full_pos_edge_index,add_matrix2),1)

In [26]:
data.train_pos_edge_index.shape

torch.Size([2, 6374])

In [27]:
data.full_pos_edge_index=full_pos_edge_index

Convert the graph to undirected graph and prepare the mask

In [28]:
dr_full_mask = torch.ones(data.full_pos_edge_index.shape[1], dtype=torch.bool)
dr_full_mask[df_global_idx] = False
dr_full_mask=dr_full_mask.to(device)

df_full_mask = torch.zeros(data.full_pos_edge_index.shape[1], dtype=torch.bool)
df_full_mask[df_global_idx] = True
df_full_mask=df_full_mask.to(device)

In [29]:
data.full_pos_edge_index.shape

torch.Size([2, 6684])

In [30]:
# Edges in S_Df
_, two_hop_edge, _, two_hop_mask = k_hop_subgraph(
        data.full_pos_edge_index[:, df_full_mask].flatten().unique(), 
        2, 
        data.full_pos_edge_index,
        num_nodes=data.num_nodes)
data.sdf_mask = two_hop_mask

In [31]:
# Nodes in S_Df
_, one_hop_edge, _, one_hop_mask = k_hop_subgraph(
    data.full_pos_edge_index[:, df_full_mask].flatten().unique(), 
    1, 
    data.full_pos_edge_index,
    num_nodes=data.num_nodes)

In [32]:
sdf_node_1hop = torch.zeros(data.num_nodes, dtype=torch.bool)
sdf_node_2hop = torch.zeros(data.num_nodes, dtype=torch.bool)

In [33]:
sdf_node_1hop[one_hop_edge.flatten().unique()] = True
sdf_node_2hop[two_hop_edge.flatten().unique()] = True
assert sdf_node_1hop.sum() == len(one_hop_edge.flatten().unique())
assert sdf_node_2hop.sum() == len(two_hop_edge.flatten().unique())

In [34]:
data.sdf_node_1hop_mask = sdf_node_1hop
data.sdf_node_2hop_mask = sdf_node_2hop

In [35]:
full_pos_edge_index1, [df_full_mask1, two_hop_mask1] = to_undirected(data.full_pos_edge_index, [df_full_mask.int(), two_hop_mask.int()])
two_hop_mask1 = two_hop_mask1.bool()
df_full_mask1 = df_full_mask1.bool()
dr_full_mask1 = ~df_full_mask

data.full_pos_edge_index1 =full_pos_edge_index1
data.edge_index1 = full_pos_edge_index1
assert is_undirected(data.full_pos_edge_index1)

In [36]:
data.sdf_mask = two_hop_mask1
data.df_full_mask = df_full_mask1
data.dr_full_mask = dr_full_mask1

Learnable edge weight

In [37]:
model_ckpt = torch.load(os.path.join(args.checkpoint_dir, 'model_final.pt'), map_location=device)
model_ori=GCN(args)
model_ori.load_state_dict(model_ckpt['model_state'], strict=False)
model_ori=model_ori.to(device)
model_ori.eval()

GCN(
  (conv1): GCNConv(3703, 128)
  (conv2): GCNConv(128, 64)
)

In [38]:
z_ori= model_ori(data.x, data.full_pos_edge_index1[:, data.sdf_mask],return_all_emb=False)
z_ori.detach()
z_ori.shape

torch.Size([3327, 64])

In [39]:
mask = PGNNMask(data.x.cuda(), n_hidden=64, temperature=1).to(device)
mask=mask.to(device)

In [65]:
optimizer_mask = torch.optim.Adam(mask.parameters(), lr = 1e-3)

In [41]:
###Delete

In [42]:
model = WGCNDelete(args, sdf_node_1hop, sdf_node_2hop, num_nodes=data.num_nodes, num_edge_type=args.num_edge_type)
model_ckpt = torch.load(os.path.join(args.checkpoint_dir, 'model_final.pt'), map_location=device)
model.load_state_dict(model_ckpt['model_state'], strict=False)
model = model.to(device)

In [43]:
parameters_to_optimize = [
                {'params': [p for n, p in model.named_parameters() if 'del' in n], 'weight_decay': 0.0}
            ]
print('parameters_to_optimize', [n for n, p in model.named_parameters() if 'del' in n])

parameters_to_optimize ['deletion1.deletion_weight', 'deletion2.deletion_weight']


In [44]:
optimizer_mlp = torch.optim.Adam(parameters_to_optimize, lr=0.00001)#, weight_decay=args.weight_decay)

In [45]:
sdf1_all_pair_mask = torch.zeros(data.num_nodes, data.num_nodes, dtype=torch.bool)
idx = torch.combinations(torch.arange(data.num_nodes)[data.sdf_node_1hop_mask], with_replacement=True).t()
sdf1_all_pair_mask[idx[0], idx[1]] = True
sdf1_all_pair_mask[idx[1], idx[0]] = True

In [46]:
assert sdf1_all_pair_mask.sum().cpu() == data.sdf_node_1hop_mask.sum().cpu() * data.sdf_node_1hop_mask.sum().cpu()

In [47]:
 ## Remove Df itself
sdf1_all_pair_mask[data.full_pos_edge_index1[:, data.df_full_mask][0], data.full_pos_edge_index1[:, data.df_full_mask][1]] = False
sdf1_all_pair_mask[data.full_pos_edge_index1[:, data.df_full_mask][1], data.full_pos_edge_index1[:, data.df_full_mask][0]] = False
##sdf1_all_pair_mask contain

In [48]:
sdf2_all_pair_mask = torch.zeros(data.num_nodes, data.num_nodes, dtype=torch.bool)
idx = torch.combinations(torch.arange(data.num_nodes)[data.sdf_node_2hop_mask], with_replacement=True).t()
sdf2_all_pair_mask[idx[0], idx[1]] = True
sdf2_all_pair_mask[idx[1], idx[0]] = True
assert sdf2_all_pair_mask.sum().cpu() == data.sdf_node_2hop_mask.sum().cpu() * data.sdf_node_2hop_mask.sum().cpu()

In [49]:
 ## Remove Df itself
sdf2_all_pair_mask[data.full_pos_edge_index1[:, data.df_full_mask][0], data.full_pos_edge_index1[:, data.df_full_mask][1]] = False
sdf2_all_pair_mask[data.full_pos_edge_index1[:, data.df_full_mask][1], data.full_pos_edge_index1[:, data.df_full_mask][0]] = False
##sdf1_all_pair_mask contain

In [50]:
 ## Lower triangular mask
idx = torch.tril_indices(data.num_nodes, data.num_nodes, -1)
lower_mask = torch.zeros(data.num_nodes, data.num_nodes, dtype=torch.bool)
lower_mask[idx[0], idx[1]] = True

In [51]:
## The final mask is the intersection
sdf1_all_pair_without_df_mask = sdf1_all_pair_mask & lower_mask
sdf2_all_pair_without_df_mask = sdf2_all_pair_mask & lower_mask

In [52]:
data

Data(x=[3327, 3703], val_pos_edge_index=[2, 455], test_pos_edge_index=[2, 910], train_pos_edge_index=[2, 6374], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 455], test_neg_edge_index=[2, 910], full_pos_edge_index=[2, 6684], sdf_mask=[6470], sdf_node_1hop_mask=[3327], sdf_node_2hop_mask=[3327], full_pos_edge_index1=[2, 6470], edge_index1=[2, 6470], df_full_mask=[6470], dr_full_mask=[6684])

In [53]:
non_df_node_mask = torch.ones(data.x.shape[0], dtype=torch.bool, device=data.x.device)
non_df_node_mask[data.full_pos_edge_index1[:,data.df_full_mask].flatten().unique()] = False

data.sdf_node_1hop_mask_non_df_mask = data.sdf_node_1hop_mask.to(device) & non_df_node_mask
data.sdf_node_2hop_mask_non_df_mask = data.sdf_node_2hop_mask.to(device) & non_df_node_mask

In [54]:
loss_fct = nn.MSELoss()

In [55]:
z_ori= model_ori(data.x, data.full_pos_edge_index1[:, data.sdf_mask],return_all_emb=False)
logits_ori=(z_ori @ z_ori.t())
logits_ori=logits_ori.detach()

In [56]:
neg_size = data.df_full_mask.sum()
neg_edge = negative_sampling(
    edge_index=data.full_pos_edge_index1,
    num_nodes=data.num_nodes,
    num_neg_samples=neg_size)

In [57]:
def eval_val(model,weight):
    model.eval()
    weight.detach()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f"{prefix}_pos_edge_index"]
        neg_edge_index = data[f"{prefix}_neg_edge_index"]
        with torch.no_grad():
            z = model(data.x, data.full_pos_edge_index1,weight)
            link_logits = model.decode(z, pos_edge_index, neg_edge_index)
        link_probs = link_logits.sigmoid()
        link_labels = get_link_labels(pos_edge_index, neg_edge_index)
        auc = roc_auc_score(link_labels.cpu(), link_probs.cpu())
        perfs.append(auc)
    return perfs

In [58]:
def eval_forget(model,weight):
    model.eval()
    weight.detach()
    neg_edge_index=data.full_pos_edge_index1[:,data.df_full_mask]
    pos_edge_index=data.full_pos_edge_index1[:,same[torch.randperm(same.shape[0])[:edge_to_delete]]]
    with torch.no_grad():
        z = model(data.x, data.full_pos_edge_index1,weight)
        link_logits = model.decode(z, pos_edge_index, neg_edge_index)
    link_probs = link_logits.sigmoid()
    link_labels = get_link_labels(pos_edge_index, neg_edge_index)
    auc = roc_auc_score(link_labels.cpu(), link_probs.cpu())
    return auc
    

In [59]:
ckpt = {
            'model_state': model.state_dict(),
            'optimizer_state': optimizer_mlp.state_dict(),
        }
best_forget=1
best_auc=0
select="forget"

In [60]:
data.full_pos_edge_index1.shape

torch.Size([2, 6470])

In [61]:
data.sdf_mask.shape

torch.Size([6470])

In [62]:
data.x.requires_grad

False

In [67]:
for epoch in range(1000):
    model.train()
    adj_sampled,adj_logits = mask(z_ori.cuda())
    weight_mask=adj_sampled[data.full_pos_edge_index1[0],data.full_pos_edge_index1[1]]
    weight=torch.ones(data.full_pos_edge_index1.shape[1]).to(device)-weight_mask
    z = model(data.x, data.full_pos_edge_index1,weight)
    neg_size = data.df_full_mask.sum()
    neg_edge_index = negative_sampling(
    edge_index=data.full_pos_edge_index1,
        num_nodes=data.num_nodes,
        num_neg_samples=neg_size)
    df_logits = model.decode(z, data.full_pos_edge_index1[:, data.df_full_mask], neg_edge_index)
    loss_size=torch.sum(weight_mask)
    loss_r = loss_fct(df_logits[:neg_size], df_logits[neg_size:])
    if sdf2_all_pair_without_df_mask.sum() != 0:
        logits_sdf = (z @ z.t())[sdf2_all_pair_without_df_mask].sigmoid()
        loss_l = loss_fct(logits_sdf, logits_ori[sdf2_all_pair_without_df_mask].sigmoid())
    alpha = 0.9
    beta=3
    loss = alpha * loss_r + (1 - alpha) * loss_l+beta*loss_size
    loss.backward(retain_graph=True)
    # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer_mlp.step()
    optimizer_mlp.zero_grad()
    optimizer_mask.step()
    optimizer_mask.zero_grad()
    
    step_log = {
        'Epoch': epoch,
        'train_loss': loss.item(),
        'loss_r': loss_r.item(),
        'loss_l': loss_l.item(),
        'loss_size': loss_size.item(),
    }

    if(epoch%100==0):
        print(step_log)
    if(epoch%100==0):
        p=eval_val(model,weight)
        val_perf, tmp_test_perf = p
        print("val",val_perf,"test",tmp_test_perf)
        auc_forget=eval_forget(model,weight)
        print("forget",auc_forget)
        #print("best_forget",best_forget)
        if(select=="forget"):
            if(auc_forget<best_forget):
                best_forget=auc_forget
                ckpt = {
                'model_state': model.state_dict(),
                'optimizer_state': optimizer_mlp.state_dict(),
            }
                torch.save(ckpt, os.path.join(args.checkpoint_dir, 'model_delete.pt'))
                print("savebyforget")
        else:
            if(tmp_test_perf>best_auc):
                tmp_test_perf>best_auc
                best_auc=tmp_test_perf
                ckpt = {
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
            }
                #torch.save(ckpt, os.path.join(args.checkpoint_dir, 'model_delete.pt'))
                print("savebyauc")
    #wandb.log(step_log)


{'Epoch': 0, 'train_loss': 6.146191596984863, 'loss_r': 0.34305331110954285, 'loss_l': 0.03359745070338249, 'loss_size': 1.9446946382522583}
val 0.7465716700881535 test 0.7193134887090931
forget 0.6660999999999999
{'Epoch': 100, 'train_loss': 6.006510257720947, 'loss_r': 0.18785344064235687, 'loss_l': 0.033580198884010315, 'loss_size': 1.9446946382522583}
val 0.7489288733244777 test 0.7216966549933583
forget 0.7056000000000001
{'Epoch': 200, 'train_loss': 6.053574562072754, 'loss_r': 0.24014875292778015, 'loss_l': 0.03356415033340454, 'loss_size': 1.9446946382522583}
val 0.747716459364811 test 0.7214847240671416
forget 0.6542999999999999
savebyforget
{'Epoch': 300, 'train_loss': 6.041684627532959, 'loss_r': 0.2269379049539566, 'loss_l': 0.033563654869794846, 'loss_size': 1.9446946382522583}
val 0.7478782755705832 test 0.7224091293322064
forget 0.7245999999999999
{'Epoch': 400, 'train_loss': 6.05255651473999, 'loss_r': 0.2390197068452835, 'loss_l': 0.03354578837752342, 'loss_size': 1.94

In [59]:
full_pos_edge_index.shape

torch.Size([2, 6684])

In [60]:
full_pos_edge_index[:,199]

tensor([ 95, 822], device='cuda:0')

In [61]:
data.train_pos_edge_index[:,199]

tensor([ 95, 822], device='cuda:0')

In [30]:
two_hop_mask.int()

tensor([0, 0, 1,  ..., 1, 0, 0], device='cuda:0', dtype=torch.int32)

In [31]:
train_pos_edge_index, [df_mask, two_hop_mask] = to_undirected(data.train_pos_edge_index, [df_mask.int(), two_hop_mask.int()])
two_hop_mask = two_hop_mask.bool()
df_mask = df_mask.bool()
dr_mask = ~df_mask

data.train_pos_edge_index = train_pos_edge_index
data.edge_index = train_pos_edge_index
assert is_undirected(data.train_pos_edge_index)

In [32]:
data.sdf_mask = two_hop_mask
data.df_mask = df_mask
data.dr_mask = dr_mask

In [8]:
delta = 0.16

for random_seed in test_seeds:

    np.random.seed(random_seed)
    data = dataset[0]
    protected_attribute = data.y
    data.train_mask = data.val_mask = data.test_mask = data.y = None
    data = train_test_split_edges(data, val_ratio=0.1, test_ratio=0.2)
    data = data.to(device)

    num_classes = len(np.unique(protected_attribute))
    N = data.num_nodes
    
    
    epochs = 101
    model = GCN(data.num_features, 128).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    

    Y = torch.LongTensor(protected_attribute).to(device)
    Y_aux = (
        Y[data.train_pos_edge_index[0, :]] != Y[data.train_pos_edge_index[1, :]]
    ).to(device)
    randomization = (
        torch.FloatTensor(epochs, Y_aux.size(0)).uniform_() < 0.5 + delta
    ).to(device)
    
    
    best_val_perf = test_perf = 0
    for epoch in range(1, epochs):
        # TRAINING    
        neg_edges_tr = negative_sampling(
            edge_index=data.train_pos_edge_index,
            num_nodes=N,
            num_neg_samples=data.train_pos_edge_index.size(1) // 2,
        ).to(device)

        if epoch == 1 or epoch % 10 == 0:
            keep = torch.where(randomization[epoch], Y_aux, ~Y_aux)

        model.train()
        optimizer.zero_grad()

        z = model.encode(data.x, data.train_pos_edge_index[:, keep])
        link_logits, _ = model.decode(
            z, data.train_pos_edge_index[:, keep], neg_edges_tr
        )
        tr_labels = get_link_labels(
            data.train_pos_edge_index[:, keep], neg_edges_tr
        ).to(device)
        
        loss = F.binary_cross_entropy_with_logits(link_logits, tr_labels)
        loss.backward()
        optimizer.step()

        # EVALUATION
        model.eval()
        perfs = []
        for prefix in ["val", "test"]:
            pos_edge_index = data[f"{prefix}_pos_edge_index"]
            neg_edge_index = data[f"{prefix}_neg_edge_index"]
            with torch.no_grad():
                z = model.encode(data.x, data.train_pos_edge_index)
                link_logits, edge_idx = model.decode(z, pos_edge_index, neg_edge_index)
            link_probs = link_logits.sigmoid()
            link_labels = get_link_labels(pos_edge_index, neg_edge_index)
            auc = roc_auc_score(link_labels.cpu(), link_probs.cpu())
            perfs.append(auc)

        val_perf, tmp_test_perf = perfs
        if val_perf > best_val_perf:
            best_val_perf = val_perf
            test_perf = tmp_test_perf
        if epoch%10==0:
            log = "Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}"
            print(log.format(epoch, loss, best_val_perf, test_perf))

    # FAIRNESS
    auc = test_perf
    cut = [0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75]
    best_acc = 0
    best_cut = 0.5
    for i in cut:
        acc = accuracy_score(link_labels.cpu(), link_probs.cpu() >= i)
        if acc > best_acc:
            best_acc = acc
            best_cut = i
    f = prediction_fairness(
        edge_idx.cpu(), link_labels.cpu(), link_probs.cpu() >= best_cut, Y.cpu()
    )
    acc_auc.append([best_acc * 100, auc * 100])
    fairness.append([x * 100 for x in f])

/opt/conda/envs/graph/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch: 010, Loss: 0.6316, Val: 0.8095, Test: 0.8118


KeyboardInterrupt: 

In [ ]:
ma = np.mean(np.asarray(acc_auc), axis=0)
mf = np.mean(np.asarray(fairness), axis=0)

sa = np.std(np.asarray(acc_auc), axis=0)
sf = np.std(np.asarray(fairness), axis=0)

print(f"ACC: {ma[0]:2f} +- {sa[0]:2f}")
print(f"AUC: {ma[1]:2f} +- {sa[1]:2f}")

print(f"DP mix: {mf[0]:2f} +- {sf[0]:2f}")
print(f"EoP mix: {mf[1]:2f} +- {sf[1]:2f}")
print(f"DP group: {mf[2]:2f} +- {sf[2]:2f}")
print(f"EoP group: {mf[3]:2f} +- {sf[3]:2f}")
print(f"DP sub: {mf[4]:2f} +- {sf[4]:2f}")
print(f"EoP sub: {mf[5]:2f} +- {sf[5]:2f}")